In [9]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [8]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Using cached openmeteo_requests-1.3.0-py3-none-any.whl.metadata (9.7 kB)
  Using cached openmeteo_sdk-1.14.1-py3-none-any.whl.metadata (934 bytes)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
Using cached openmeteo_requests-1.3.0-py3-none-any.whl (6.0 kB)
Using cached openmeteo_sdk-1.14.1-py3-none-any.whl (7.3 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached cattrs-24.1.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached requests_cache-1.2.1-py3-none-any.whl (61 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
Using cached cattrs-24.1.0-py3-none-any.whl (66 kB)
Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# API Call

In [10]:
def prediction_call(locations):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": locations[0],
        "longitude": locations[1],
        "daily": ["temperature_2m_max","temperature_2m_min","rain_sum",],
        "Timezone": "auto"
    }
    responses = openmeteo.weather_api(url, params=params)

    return responses

# Process API Call

In [11]:
def extract_process(responses):
    response = responses[0]

    daily_data = response.Daily()

    temp_min = daily_data.Variables(0).ValuesAsNumpy()
    temp_max = daily_data.Variables(1).ValuesAsNumpy()
    rain_sum = daily_data.Variables(2).ValuesAsNumpy()
    response = responses[0]

    daily_data = response.Daily()
    daily_index = pd.date_range(
            start = pd.to_datetime(daily_data.Time(), unit = "s"),
            end = pd.to_datetime(daily_data.TimeEnd(), unit = "s"),
            freq = pd.Timedelta(seconds = daily_data.Interval()),
            inclusive = "left"
            )
    temp_min = daily_data.Variables(0).ValuesAsNumpy()
    temp_max = daily_data.Variables(1).ValuesAsNumpy()
    rain_sum = daily_data.Variables(2).ValuesAsNumpy()
    daily_df = {
            "Datetime" : daily_index,
            "Minimum Temperature" : temp_min,
            "Maximum Temperature" : temp_max,
            "Rain" : rain_sum
        }

    daily_df = pd.DataFrame(daily_df)
    return daily_df

    
    

In [12]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
locations = [52,64]
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": locations[0],
    "longitude": locations[1],
    "daily": ["temperature_2m_max","temperature_2m_min","rain_sum",],
    "Timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

daily_data = response.Daily()
daily_index = pd.date_range(
        start = pd.to_datetime(daily_data.Time(), unit = "s"),
        end = pd.to_datetime(daily_data.TimeEnd(), unit = "s"),
        freq = pd.Timedelta(seconds = daily_data.Interval()),
        inclusive = "left"
        )
temp_min = daily_data.Variables(0).ValuesAsNumpy()
temp_max = daily_data.Variables(1).ValuesAsNumpy()
rain_sum = daily_data.Variables(2).ValuesAsNumpy()
daily_df = {
        "Datetime" : daily_index,
        "Minimum Temperature" : temp_min,
        "Maximum Temperature" : temp_max,
        "Rain" : rain_sum
    }

daily_df = pd.DataFrame(daily_df)
daily_df


,Datetime,Minimum Temperature,Maximum Temperature,Rain
0,2024-09-03,23.174002,8.874,4.3
1,2024-09-04,14.574000,7.724,0.0
2,2024-09-05,12.374000,6.024,0.0
3,2024-09-06,13.774000,5.874,0.0
4,2024-09-07,17.524000,6.674,0.0
5,2024-09-08,21.374001,9.924,0.0
6,2024-09-09,22.824001,10.124,0.0
